In [ ]:
import xarray as xr
import os
import numpy as np
from datetime import datetime as dt
from HydroViewer import selectHydroViewer

hydro_viewer = selectHydroViewer.hydro_viewer
%matplotlib inline 

In [ ]:
flight = '20150706'
probeName = 'CIP'
campaign = 'PECAN'
runLoc = 'local' # or 'keeling'

if runLoc is 'local':
    dataDir = '/Users/danstechman/GoogleDrive/PECAN-Data/'
    plotDir = dataDir + 'mp-data/' + flight + '/binTemps_imgDump/'
    imgFile = dataDir + 'mp-data/' + flight + '/DIMG.' + probeName + '.' + flight + '.cdf'
    pbpFile = dataDir + 'mp-data/' + flight + '/proc2.' + flight + '.' + probeName + '.cdf'
    flFile = dataDir + 'FlightLevelData/Processed/' + flight + '_FltLvl_Processed.nc'
elif runLoc is 'keeling':
    dataDir = '/data/pecan/a/stechma2/pecan/'
    plotDir = dataDir + 'mp-data/UIOPS/' + flight + '/binTemps_imgDump/'
    imgFile = dataDir + 'mp-data/UIOPS/' + flight + '/DIMG.' + probeName + '.' + flight + '.cdf'
    pbpFile = dataDir + 'mp-data/UIOPS/' + flight + '/proc2.' + flight + '.' + probeName + '.cdf'
    flFile = dataDir + 'flight-level-data/processed/' + flight + '_FltLvl_Processed.nc'
    
if not os.path.exists(plotDir):
    os.makedirs(plotDir)

pddSprlTimes = np.load(dataDir + 'pecan_PDD-Sprl_times/' + flight + '_pddSprl_times.npy')[()]

flData = xr.open_dataset(flFile)
temp = flData.get('TA').to_masked_array()
dtNum = flData.get('datetime_FL').data
dtStr = np.char.mod('%d',dtNum)
flDT = np.asarray([dt.strptime(fDate,'%Y%m%d%H%M%S') for fDate in dtStr])

In [ ]:
if flight is '20150617':
        sprlS = pddSprlTimes['I11S_strtDT']
        sprlE = pddSprlTimes['I11S_endDT']
    
        tempBinEdges = np.arange(-19.5,21.5,1.0)

    elif flight is '20150620':
        sprlS = pddSprlTimes['U4S_strtDT']
        sprlE = pddSprlTimes['U4S_endDT']
    
        tempBinEdges = np.arange(-19.5,21.5,1.0)
        
    elif flight is '20150701':
        sprlS = pddSprlTimes['I17S_strtDT']
        sprlE = pddSprlTimes['I17S_endDT']
    
        tempBinEdges = np.arange(-19.5,21.5,1.0)
    
    elif flight is '20150702':
        sprlS = pddSprlTimes['U8AS_strtDT']
        sprlE = pddSprlTimes['U8AS_endDT']
    
        tempBinEdges = np.arange(-19.5,22.5,1.0)
    
    elif flight is '20150706':
        sprlS = pddSprlTimes['I20S_strtDT']
        sprlE = pddSprlTimes['I20S_endDT']
    
        tempBinEdges = np.arange(-19.5,20.5,1.0)
        
    elif flight is '20150709':
        sprlS = pddSprlTimes['I21S_strtDT']
        sprlE = pddSprlTimes['I21S_endDT']
    
        tempBinEdges = np.arange(-19.5,21.5,1.0)

In [ ]:
for sprl in range(0,len(sprlS)):
    print('\n----- Working on Spiral {} of {} -----'.format(sprl+1,len(sprlS)))
    strtIx = np.squeeze(np.where(flDT == sprlS[sprl]))
    endIx = np.squeeze(np.where(flDT == sprlE[sprl]))
    dtSprl = flDT[strtIx:endIx]
    tempSprl = temp[strtIx:endIx]
    
    histIxs = np.digitize(tempSprl,bins=tempBinEdges)
    for binIx in range(1,len(tempBinEdges)):
        print('Temperature bin = {}C - {}C'.format(tempBinEdges[binIx-1],tempBinEdges[binIx]))
        binnedTimes = dtSprl[np.where(histIxs == binIx)]
        if len(binnedTimes) == 0:
            print('\tNo particles observed in this temperature range')
            continue
        else:
            print('\tFound {} frames in this temperature range'.format(len(binnedTimes)))
        outDT = []
        for dtime in binnedTimes:
            outDT.append(int(dt.strftime(dtime,'%H%M%S')))
        filePrfx = '{}_S{:02d}_{:02d}C_'.format(flight,sprl+1,int(np.mean(tempBinEdges[binIx-1:binIx+1])))
        hydro_viewer(imgFile,pbpFile,plotDir,campaign,probeName,outDT,filePrfx=filePrfx)